# Testing newest implementation of groups assigment

In [ ]:
import os
import sys
import plotly.express as px
import plotly.graph_objects as go


module_path = os.path.abspath(os.path.join('./../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.divergence_matrix.DivergenceMatrixProcessor import DivergenceMatrixProcessor
from src.state_comparator.comparator_functions import *

from src.helper_functions import pretty_print
import src.configfile as config

instance = DivergenceMatrixProcessor("./../../data/divergence_matrix/Divergence_M.pickle")

In [ ]:
leak_amt = 16.0
sensor_node = "Jonctiune-3974"
# descending_values, jenks_natural_breaks
groups_dict = instance.get_affected_nodes_groups(16.0, sensor_node, num_of_groups=3, method="jenks_natural_breaks")

print("Dict of nodes and their influence group")
groups_dict

## Optimal number of groups

In [ ]:
from jenkspy import jenks_breaks
import numpy as np

def goodness_of_variance_fit(array, classes):
    # get the break points
    classes = jenks_breaks(array, classes)

    # do the actual classification
    classified = np.array([classify(i, classes) for i in array])

    # max value of zones
    maxz = max(classified)

    # nested list of zone indices
    zone_indices = [[idx for idx, val in enumerate(classified) if zone + 1 == val] for zone in range(maxz)]

    # sum of squared deviations from array mean
    sdam = np.sum((array - array.mean()) ** 2)

    # sorted polygon stats
    array_sort = [np.array([array[index] for index in zone]) for zone in zone_indices]

    # sum of squared deviations of class means
    sdcm = sum([np.sum((classified - classified.mean()) ** 2) for classified in array_sort])

    # goodness of variance fit
    gvf = (sdam - sdcm) / sdam

    return gvf

def classify(value, breaks):
    for i in range(1, len(breaks)):
        if value < breaks[i]:
            return i
    return len(breaks) - 1

In [ ]:
# ["SenzorComunarzi-NatVech", "SenzorCernauti-Sebesului", "SenzorChisinau-Titulescu",
#                 "SenzorComunarzi-castanului", "Jonctiune-3974", "Jonctiune-J-3", "Jonctiune-J-19", "Jonctiune-2749"]
sensor_node = "SenzorComunarzi-NatVech"
time_stamp = 36000
series_at_timestamp = instance.extract_df_with_specific_leak_on_one_node(leak_amt, sensor_node)[time_stamp]
sorted_df_at_timestamp = series_at_timestamp.sort_values(ascending=True).reset_index()
series_len = len(sorted_df_at_timestamp)
sorted_arr = sorted_df_at_timestamp[time_stamp].to_numpy()
# display(sorted_arr)

In [ ]:
gvf = 0.0
nclasses = 2
while nclasses < 20:
    gvf = goodness_of_variance_fit(sorted_arr, nclasses)
    print("classes ", nclasses, "gvf", gvf)
    nclasses += 1